<a href="https://colab.research.google.com/github/seismosmsr/machine_learning/blob/main/summarize-chatgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# for google colab you have to install this 2 library before run the code
!pip install pypdf2
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [69]:
import os
import PyPDF2
import re
import openai
import pandas as pd
import json
import csv
import json


In [52]:
def merge_csvs_from_directory(directory_path, output_filename, starting_string, failed_filename):
    all_dataframes = []
    failed_files = []
    differing_structure_files = []

    first_file = True
    columns_of_first_file = None

    for filename in os.listdir(directory_path):
        if filename.startswith(starting_string) and filename.endswith('.csv'):
            filepath = os.path.join(directory_path, filename)
            try:
                df = pd.read_csv(filepath)

                # If it's the first file, save its columns
                if first_file:
                    columns_of_first_file = df.columns.tolist()
                    first_file = False

                # If columns don't match, skip this file
                if df.columns.tolist() != columns_of_first_file:
                    differing_structure_files.append(filename)
                else:
                    all_dataframes.append(df)
            except Exception as e:
                failed_files.append({'filename': filename, 'error': str(e)})
                print(f"Failed to read {filename} due to error: {e}")

    # Concatenate all successfully read dataframes with the same structure
    if all_dataframes:
        merged_df = pd.concat(all_dataframes, ignore_index=True)
        merged_df.to_csv(output_filename, index=False)

    # Compile failures and differing structures into a single list
    for file in differing_structure_files:
        failed_files.append({'filename': file, 'error': 'Differing structure'})

    # Write failed files to a separate CSV
    if failed_files:
        failed_df = pd.DataFrame(failed_files)
        failed_df.to_csv(failed_filename, index=False)

# Example usage
# merge_csvs_from_directory('/path/to/csv/directory', 'merged_output.csv', 'failed_files.csv')


In [28]:
import zipfile
import os

def zip_specific_csv_files_in_directory(directory_path, output_zipfile, starting_string):
    with zipfile.ZipFile(output_zipfile, 'w') as zipf:
        for filename in os.listdir(directory_path):
            if filename.startswith(starting_string) and filename.endswith('.csv'):
                filepath = os.path.join(directory_path, filename)
                if os.path.isfile(filepath):  # Ensure it's a file and not a directory
                    zipf.write(filepath, os.path.basename(filepath))  # Only write the base filename

    print(f"CSV files in '{directory_path}' starting with '{starting_string}' zipped to '{output_zipfile}'")

# Example usage


In [29]:
import zipfile
import os

def zip_selected_files(directory_path, output_zipfile, file_list):
    with zipfile.ZipFile(output_zipfile, 'w') as zipf:
        for filename in file_list:
            filepath = os.path.join(directory_path, filename)
            if os.path.isfile(filepath):  # Ensure it's a file and not a directory
                zipf.write(filepath, os.path.basename(filepath))

    print(f"Selected files from '{directory_path}' zipped to '{output_zipfile}'")

# Example usage
# file_list = ['file1.txt', 'file2.csv', 'image.jpg']


In [30]:
import os
import pandas as pd

def is_csv_empty(file_path):
    return os.path.getsize(file_path) == 0

def has_uneven_columns(df):
    max_cols = len(df.columns)
    for index, row in df.iterrows():
        if len(row) != max_cols:
            return True
    return False

def fix_uneven_columns(df):
    max_cols = len(df.columns)
    mask = df.apply(len) != max_cols
    rows_with_issues = df[mask].index.tolist()

    # Drop rows with issues
    df = df.drop(rows_with_issues)
    return df


In [44]:
# Download pdf direcly
!curl -o paper.pdf https://anr.vermont.gov/sites/anr/files/specialtopics/climate/documents/VTCCwhitepapers/VTCCAdaptAgriculture.pdf #change the url with pdf which you wanna download

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  416k  100  416k    0     0   317k      0  0:00:01  0:00:01 --:--:--  317k


In [45]:
# Set the string that will contain the summary
pdf_summary_text = ""

# Open the PDF file
pdf_file_path = "paper.pdf"

# Read the PDF file using PyPDF2
pdf_file = open(pdf_file_path, 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)

In [46]:
openai.api_key = "na" #change the api key with yours

In [48]:
# pdf_reader.pages[5].extract_text()
range(len(pdf_reader.pages))[0:]

range(0, 6)

In [66]:
pdf_summary_text = ""
# Loop through all the pages in the PDF file
# range(len(pdf_reader.pages))
for page_num in range(len(pdf_reader.pages)):
    # Extract the text from the page
    page_text = pdf_reader.pages[page_num].extract_text().lower()
    # print(page_text)
    bad_read = True
    attempts = 0
    while bad_read:
      response = openai.ChatCompletion.create(
                      model="gpt-3.5-turbo",
                      messages=[
                          {"role": "system", "content": "You are a helpful research assistant. Specfically, you are helping research the impact of climate change on global food systems. We are doing a literature review. We want to know what crops and food systems are being impacted by specific climate change hazards. Only respond in the form of comma seperated values (csv). You always return a correctly formatted csv."},
                          {"role": "user", "content": f"Summarize the following text as a JSON document. Do not reply with anything except a JSON document. Please identify any specific climate change hazards, what cropping or food systems they effect, where globally the impact will be experienced, the specific quote from the paper (no more than 100 characters), if the impact is generally positive or negative (sentiment), and approximately what magnitude (e.g. high medium low). Do not do anything that could possibly break JSON formatting. Please make sure that text entries do not use commas internal to any text entries in the table. Please only return a JSON. The elements should only be: region, cropping_system, impact, sentiment, magnitude, quote, page_number (you are working on page {page_num}), paragraph_number. Here is the text:{page_text}. Do not return anything but the properly formatted JSON. It is of the utmost importance that the response you give is a properly formatted JSON."},
                          ])
      page_summary = response["choices"][0]["message"]["content"]

      pdf_summary_text+=page_summary + "\n"

      pdf_summary_file = pdf_file_path.replace(os.path.splitext(pdf_file_path)[1], f"_summary.txt")

      with open(pdf_summary_file, "w+") as file:
          file.write(pdf_summary_text)

      pdf_summary_file = pdf_file_path.replace(os.path.splitext(pdf_file_path)[1], f"_{page_num}_summary.json")

      with open(pdf_summary_file, "w+") as file:
          file.write(page_summary)

      # file_path = 'path_to_your_csv_file.csv'
      attempts = attempts + 1
      # Check if the CSV is empty
      # if is_csv_empty(pdf_summary_file):
      #     print(f"{pdf_summary_file} is empty.")
      # else:
      #     try:
      #         df = pd.read_csv(pdf_summary_file)

      #         # If the CSV has uneven columns, fix it
      #         if has_uneven_columns(df):
      #             # print("Fixing uneven columns...")
      #             df = fix_uneven_columns(df)
      #             df.to_csv(pdf_summary_file, index=False)  # overwrite the original file with the fixed data
      #             bad_read = False

      #         else:
      #             print("No issues detected. File remains unchanged.")
      #             bad_read = False


      #     except Exception as e:
      #         print(f"Error reading file {pdf_summary_file}: {e}")
      #         print(f"Attempt {attempts} failed.")
      bad_read = False





# Close the PDF file
pdf_file.close()

with open(pdf_summary_file, "r") as file:
    print(file.read())

{
  "region": "U.S. Northeast",
  "cropping_system": "Agriculture",
  "impact": "Climate Change",
  "sentiment": "Negative",
  "magnitude": "High",
  "quote": "Climate change impacts on northeastern agriculture: an overview.",
  "page_number": "5",
  "paragraph_number": "5"
}


In [57]:
df = pd.read_csv('/content/paper_0_summary.csv')
# df

In [73]:
all_data = []
# Loop through all JSON files in the directory.
for filename in os.listdir('/content/'):
    if filename.endswith('.json'):
        json_file = os.path.join('/content/', filename)

        with open(json_file, 'r') as file:
            print(file)
            data = json.load(file)

            # If your JSON data is a list of objects, you may need to flatten it as mentioned in the previous response.
            # For this example, we assume each JSON file contains a list of objects.
            all_data.extend(data)

<_io.TextIOWrapper name='/content/paper_0_summary.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='/content/paper_1_summary.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='/content/paper_4_summary.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='/content/paper_5_summary.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='/content/paper_3_summary.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='/content/paper_2_summary.json' mode='r' encoding='UTF-8'>


In [74]:
all_data

['region',
 'cropping_system',
 'impact',
 'sentiment',
 'magnitude',
 'quote',
 'page_number',
 'paragraph_number',
 {'region': 'Vermont',
  'cropping_system': 'Warm-weather crops',
  'impact': 'Increase in productivity',
  'sentiment': 'Positive',
  'magnitude': 'Moderate',
  'quote': 'increase the productivity of warm-weather crops',
  'page_number': 1,
  'paragraph_number': 2},
 {'region': 'Vermont',
  'cropping_system': 'Cold-weather crops',
  'impact': 'Decrease in productivity',
  'sentiment': 'Negative',
  'magnitude': 'Moderate',
  'quote': 'cold-weather crops will be among the most vulnerable',
  'page_number': 1,
  'paragraph_number': 4},
 {'region': 'Vermont',
  'cropping_system': 'Grain crops, warm-weather crops',
  'impact': 'Reduced productivity',
  'sentiment': 'Negative',
  'magnitude': 'Low',
  'quote': 'grain crops such as field corn, wheat, and oats tend to experience lower yields when summer temperatures rise',
  'page_number': 1,
  'paragraph_number': 4},
 {'regio

In [84]:
# List of files
files = [f for f in os.listdir('/content/') if f.endswith('.json')]

# List to store dataframes
dfs = []

# Read each file and convert to dataframe
for file in files:
    with open(file, 'r') as f:
        data = json.load(f)

        # If the JSON data is a list of dictionaries
        if isinstance(data, list) and all(isinstance(item, dict) for item in data):
            df = pd.DataFrame(data)
            dfs.append(df)

        # If the JSON data is a single dictionary
        elif isinstance(data, dict):
            df = pd.DataFrame([data])
            dfs.append(df)

        # Handle other structures as needed
        else:
            print(f"Unhandled data structure in {file}. Skipping.")

# Combine all dataframes
combined_df = pd.concat(dfs, ignore_index=True)

# Write combined dataframe to CSV
combined_df.to_csv('merged_output.csv', index=False)


In [58]:
merge_csvs_from_directory('/content/', 'merged_output.csv','paper_',failed_filename='failed_files.csv')

In [51]:
zip_selected_files('/content/', 'results_3.zip', ['paper.pdf','merged_output.csv','failed_files.csv','csv_files.zip'])


Selected files from '/content/' zipped to 'results_3.zip'
